In [ ]:
%pip install -U langgraph langchain-tavily langgraph-checkpoint-sqlite

In [ ]:
!pip install openai

In [ ]:
!pip install -U jupyter-client

In [ ]:
!pip install -U langchain-openai

In [ ]:
from langgraph.prebuilt import create_react_agent

In [ ]:
from langchain_core.tools import tool

In [ ]:
pip install langchain pydantic requests


In [3]:
import os
import time
import re
import requests
from urllib.parse import urlparse, parse_qsl, urlencode, urlunparse
from typing import List, Optional, Dict, Any
from pydantic import BaseModel, Field
from google.colab import userdata
from langchain.tools import StructuredTool

GOOGLE_CSE_API_KEY = userdata.get("GOOGLE_CSE")
GOOGLE_CSE_CX = userdata.get("GOOGLE_CSE_CX")
GOOGLE_CSE_ENDPOINT = "https://www.googleapis.com/customsearch/v1"

# --- helpers ---
def now_iso() -> str:
    return time.strftime("%Y-%m-%dT%H:%M:%SZ", time.gmtime())

def canonicalize_url(url: str) -> str:
    try:
        parts = urlparse(url)
        netloc = parts.netloc.lower().replace(":80","").replace(":443","")
        q = [(k,v) for k,v in parse_qsl(parts.query) if not k.startswith("utm_") and k not in {"gclid","fbclid"}]
        return urlunparse((parts.scheme, netloc, parts.path, parts.params, urlencode(q), parts.fragment))
    except Exception:
        return url

class GoogleSearchInput(BaseModel):
    query: str = Field(..., description="Web search query")
    k: int = Field(5, ge=1, le=10, description="Max results to return")
    safe: str = Field("active", description="SafeSearch mode: active|off")
    lr: Optional[str] = Field(None, description="Language restrict, e.g. 'lang_en'")

def google_search(query: str, k: int = 5, safe: str = "active", lr: Optional[str] = None) -> List[Dict[str, Any]]:
    if not GOOGLE_CSE_API_KEY or not GOOGLE_CSE_CX:
        raise RuntimeError("Set GOOGLE_CSE_API_KEY and GOOGLE_CSE_CX env vars")

    params = {
        "key": GOOGLE_CSE_API_KEY,
        "cx": GOOGLE_CSE_CX,
        "q": query,
        "num": min(k, 10),
        "safe": safe
    }
    if lr:
        params["lr"] = lr

    resp = requests.get(GOOGLE_CSE_ENDPOINT, params=params, timeout=15)
    resp.raise_for_status()
    data = resp.json()

    hits = []
    for it in data.get("items", []):
        title = (it.get("title") or "").strip()
        url = canonicalize_url((it.get("link") or "").strip())
        snippet = (it.get("snippet") or "").strip()
        if not title or not url:
            continue
        snippet = re.sub(r"\s+", " ", snippet)
        hits.append({
            "title": title,
            "url": url,
            "snippet": snippet,
            "source": "google_cse",
            "seen_at": now_iso()
        })
    return hits[:k]

GoogleSearchTool = StructuredTool.from_function(
    func=google_search,
    name="web_search",
    description="Search return {title,url,snippet,source,seen_at}.",
    args_schema=GoogleSearchInput
)


In [4]:
if __name__ == "__main__":
    results = GoogleSearchTool.invoke({"query": "CRISPR gene editing 2023 review", "k": 3})
    for r in results:
        print(r)


{'title': 'CRISPR/Cas9 therapeutics: progress and prospects | Signal ...', 'url': 'https://www.nature.com/articles/s41392-023-01309-7', 'snippet': 'Jan 16, 2023 ... This paper reviews the current developments in three aspects, namely, gene-editing type, delivery vector, and disease characteristics.', 'source': 'google_cse', 'seen_at': '2025-08-24T01:12:08Z'}
{'title': 'Unintended CRISPR-Cas9 editing outcomes: a review of the ...', 'url': 'https://pubmed.ncbi.nlm.nih.gov/37093294/', 'snippet': '... editing applications. Here we review SVs associated with CRISPR-editing in cells of human origin and highlight the methods used to detect and avoid them. © 2023 ...', 'source': 'google_cse', 'seen_at': '2025-08-24T01:12:08Z'}
{'title': 'Application of CRISPR-Cas9 genome editing technology in various ...', 'url': 'https://pmc.ncbi.nlm.nih.gov/articles/PMC10916045/', 'snippet': 'Overall, CRISPR-Cas9 technology has revolutionized genome editing, providing researchers with a versatile and efficie

In [14]:
from langchain.agents import initialize_agent, AgentType
from langgraph.prebuilt import create_react_agent
import openai

user_input = input("What is your article about?")

perspectives = ['Molecular Biologist', 'Medical Researcher', 'Agricultural Scientist', 'Bioethicist', 'Regulatory Policy Expert', 'Basic Fact Researcher']

ASKER = init_chat_model("gpt-4o-mini", model_provider="openai", openai_api_key=userdata.get("OPENAI_API_KEY"))
LLM_2 = init_chat_model("gpt-4.1", model_provider="openai", openai_api_key=userdata.get("OPENAI_API_KEY"))
tools = [GoogleSearchTool]
EXPERT_RESEARCHER = create_react_agent(LLM_2, tools)

convo = []
M = 6
for p in perspectives:
  convo_history = []
  for i in range(M):
    query = f"You are a Wikipedia Writer and you are writing an article on {user_input}, ask a question from the perspective of a {p} to contribute to further understanding of the topic based on the given conversation history between you and an expert: {convo_history}. Only respond with the question"
    q = ASKER.invoke([{"role": "user", "content": query}])
    convo_history.append(q.content)
    input_message = {"role": "user", "content": f"Split the given query: {q} into Google searches in order to best answer the question, respond with at most 3 credible sources from the web"}
    a = EXPERT_RESEARCHER.invoke({"messages": [input_message]})
    convo_history.append(a)

  convo.append(convo_history)

print("end")

What is your article about?CRISPR
end


What is your article about? CRISPR

=== Persona: Molecular Biologist ===
[Molecular Biologist Asker] What are the most recent advancements in CRISPR technology for therapeutic applications?
[Expert] Recent advancements in CRISPR technology for therapeutic applications include:

- Expansion of CRISPR-based gene editing in clinical trials, with a notable shift in the CRISPR medicine landscape as of 2024. There is increased focus on refining delivery methods and improving safety and efficacy for human therapies (see Innovative Genomics).
- Progress in using CRISPR/Cas9 for cancer immunotherapy, though challenges remain in clinical translation, such as immune response management and precise targeting (see EHO Online).
- The use of CRISPR screens to identify new therapeutic targets, which is opening promising avenues for treatment, especially in oncology and genetic diseases (see Molecular Cancer).

These developments highlight ongoing efforts to translate CRISPR technology into effective a

/tmp/ipython-input-3874045033.py:144: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  print("\nDone at", datetime.utcnow().isoformat() + "Z")
